# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
import joblib
import numpy as np

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv('../data/day-of-week-not-scaled.csv')
df_day = pd.read_csv('../data/dayofweek.csv')
df['dayofweek'] = df_day['dayofweek']

In [3]:
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('dayofweek', axis=1), 
    df['dayofweek'],
    stratify=df['dayofweek'],
    test_size=0.2,
    random_state=21,
    shuffle=True
)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train,
    y_train,
    stratify=y_train,
    test_size=0.2,
    random_state=21,
    shuffle=True
)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [6]:
svm = SVC(kernel='rbf', 
          C=10,
          class_weight=None,
          gamma='auto',
          probability=True,
          random_state=21
          )
svm.fit(X_train, y_train)
y_pred = svm.predict(X_valid)
svm_model = svm
accuracy_svm = accuracy_score(y_valid, y_pred)
precision_svm = precision_score(y_valid, y_pred, average='weighted')
print(f"SVM:")
print(f"accuracy is {accuracy_score(y_valid, y_pred):.5f}")
print(f"precision is {precision_score(y_valid, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_valid, y_pred, average='weighted'):.5f}")

SVM:
accuracy is 0.87778
precision is 0.88162
recall is 0.87778


In [7]:
tree = DecisionTreeClassifier(
    class_weight='balanced',
    criterion='gini',
    max_depth=21,
    random_state=21
)
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_valid)
tree_model = tree
accuracy_tree = accuracy_score(y_valid, y_pred_tree)
precision_tree = precision_score(y_valid, y_pred_tree, average='weighted')
print(f"Decision Tree:")
print(f"accuracy is {accuracy_score(y_valid, y_pred_tree):.5f}")
print(f"precision is {precision_score(y_valid, y_pred_tree, average='weighted'):.5f}")
print(f"recall is {recall_score(y_valid, y_pred_tree, average='weighted'):.5f}")

Decision Tree:
accuracy is 0.86667
precision is 0.87170
recall is 0.86667


In [8]:
rfc = RandomForestClassifier(
    class_weight='balanced',
    criterion='entropy',
    max_depth=24,
    n_estimators=100,
    random_state=21
)
rfc.fit(X_train, y_train)
y_pred_rfc = rfc.predict(X_valid)
rfc_model = rfc
accuracy_rfc = accuracy_score(y_valid, y_pred_rfc)
precision_rfc = precision_score(y_valid, y_pred_rfc, average='weighted')
print(f"Random Forest:")
print(f"accuracy is {accuracy_score(y_valid, y_pred_rfc):.5f}")
print(f"precision is {precision_score(y_valid, y_pred_rfc, average='weighted'):.5f}")
print(f"recall is {recall_score(y_valid, y_pred_rfc, average='weighted'):.5f}")

Random Forest:
accuracy is 0.89630
precision is 0.89698
recall is 0.89630


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [9]:
voting_params_to_try = [
    {'voting': 'hard', 'weights': None},
    {'voting': 'soft', 'weights': None},
    {'voting': 'hard', 'weights': [1, 1, 1]},
    {'voting': 'soft', 'weights': [1, 1, 1]},
    {'voting': 'hard', 'weights': [2, 1, 2]},
    {'voting': 'soft', 'weights': [2, 1, 2]},
    {'voting': 'hard', 'weights': [3, 1, 3]},
    {'voting': 'soft', 'weights': [3, 1, 3]},
    {'voting': 'hard', 'weights': [4, 1, 4]},
    {'voting': 'soft', 'weights': [4, 1, 4]},
]

best_voting_accuracy = 0
best_voting_params = None
best_voting_model = None


for params in voting_params_to_try:
    voting_clf = VotingClassifier(
        estimators=[
            ('svm', svm),
            ('tree', tree),
            ('rf', rfc)
        ],
        voting=params['voting'],
        weights=params['weights'],
        n_jobs=-1
    )
    
    voting_clf.fit(X_train, y_train)
    y_pred = voting_clf.predict(X_valid)
    
    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
    
    print(f"voting: {params['voting']}, weights: {params['weights']}")
    print(f"accuracy: {accuracy:.5f}, precision: {precision:.5f}")
    
    if accuracy > best_voting_accuracy or (accuracy == best_voting_accuracy and precision > best_voting_precision):
        best_voting_accuracy = accuracy
        best_voting_precision = precision
        best_voting_params = params
        best_voting_model = voting_clf

print(f"Best parameters: {best_voting_params}")
print(f"Best accuracy: {best_voting_accuracy:.5f}")
print(f"Best precision: {best_voting_precision:.5f}")

voting: hard, weights: None
accuracy: 0.90000, precision: 0.89993
voting: soft, weights: None
accuracy: 0.88519, precision: 0.88840
voting: hard, weights: [1, 1, 1]
accuracy: 0.90000, precision: 0.89993
voting: soft, weights: [1, 1, 1]
accuracy: 0.88519, precision: 0.88840
voting: hard, weights: [2, 1, 2]
accuracy: 0.90000, precision: 0.89993
voting: soft, weights: [2, 1, 2]
accuracy: 0.90370, precision: 0.90462
voting: hard, weights: [3, 1, 3]
accuracy: 0.90000, precision: 0.89993
voting: soft, weights: [3, 1, 3]
accuracy: 0.90370, precision: 0.90364
voting: hard, weights: [4, 1, 4]
accuracy: 0.90000, precision: 0.89993
voting: soft, weights: [4, 1, 4]
accuracy: 0.91111, precision: 0.91288
Best parameters: {'voting': 'soft', 'weights': [4, 1, 4]}
Best accuracy: 0.91111
Best precision: 0.91288


In [10]:
y_pred_best_voting = best_voting_model.predict(X_valid)
recall_best_voting = recall_score(y_valid, y_pred_best_voting, average='weighted', zero_division=0)

print(f"Best model Voting Classifier:")
print(f"Parameters: {best_voting_params}")
print(f"Accuracy: {best_voting_accuracy:.5f}")
print(f"Precision: {best_voting_precision:.5f}")
print(f"Recall: {recall_best_voting:.5f}")

y_pred_test_voting = best_voting_model.predict(X_test)
accuracy_test_voting = accuracy_score(y_test, y_pred_test_voting)
precision_test_voting = precision_score(y_test, y_pred_test_voting, average='weighted')
recall_test_voting = recall_score(y_test, y_pred_test_voting, average='weighted', zero_division=0)

print("\nTest sample:")
print(f"accuracy is {accuracy_test_voting:.5f}")
print(f"precision is {precision_test_voting:.5f}")
print(f"recall is {recall_test_voting:.5f}")

Best model Voting Classifier:
Parameters: {'voting': 'soft', 'weights': [4, 1, 4]}
Accuracy: 0.91111
Precision: 0.91288
Recall: 0.91111

Test sample:
accuracy is 0.90533
precision is 0.90881
recall is 0.90533


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [11]:
bagging_clf = BaggingClassifier(
    base_estimator=SVC(C=10, gamma='auto', probability=True, random_state=21, kernel='rbf'),
    n_estimators=50, n_jobs=-1, random_state=21)
y_pred = bagging_clf.fit(X_train, y_train).predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.5f}")

Accuracy: 0.88462
Precision: 0.88941
Recall: 0.88462


In [12]:
bagging_clf = BaggingClassifier(
    base_estimator=SVC(C=10, gamma='auto', probability=True, random_state=21, kernel='rbf'),
    n_estimators=40, n_jobs=-1, random_state=6, bootstrap=False)
y_pred = bagging_clf.fit(X_train, y_train).predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.5f}")

Accuracy: 0.88757
Precision: 0.88906
Recall: 0.88757


In [13]:
bagging_clf = BaggingClassifier(
    base_estimator=SVC(C=10, gamma='auto', probability=True, random_state=21, kernel='rbf'),
    n_estimators=60, n_jobs=-1, random_state=21, warm_start=True, bootstrap_features=True)
y_pred = bagging_clf.fit(X_train, y_train).predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.5f}")

Accuracy: 0.79882
Precision: 0.82461
Recall: 0.79882


In [14]:
bagging_clf = BaggingClassifier(
    base_estimator=SVC(C=10, gamma='auto', probability=True, random_state=21, kernel='rbf'),
    n_estimators=20, n_jobs=-1, random_state=15, warm_start=True)
y_pred = bagging_clf.fit(X_train, y_train).predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.5f}")

Accuracy: 0.87574
Precision: 0.88351
Recall: 0.87574


In [15]:
bagging_clf = BaggingClassifier(
    base_estimator=SVC(C=10, gamma='auto', probability=True, random_state=21, kernel='rbf'),
    n_estimators=100, n_jobs=-1, oob_score=True, random_state=50)
y_pred = bagging_clf.fit(X_train, y_train).predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.5f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.5f}")

Accuracy: 0.87574
Precision: 0.88161
Recall: 0.87574


In [16]:
bagging_results = []
for n in [5, 10, 25, 50]:
    bagging = BaggingClassifier(
        SVC(
            kernel='rbf',
            C=10,
            gamma='auto',
            class_weight=None,
            probability=True,
            random_state=21
        ),
        n_estimators=n,
        random_state=21,
        n_jobs=-1
    )
    y_pred_bag = bagging.fit(X_train, y_train).predict(X_test)
    bagging_results.append({
        'n_estimators': n,
        'accuracy': accuracy_score(y_test, y_pred_bag),
        'precision': precision_score(y_test, y_pred_bag, average='weighted'),
        'recall': recall_score(y_test, y_pred_bag, average='weighted'),
        'model': bagging
    })

In [17]:
best_bagging = sorted(bagging_results, key=lambda x: (x['accuracy'], x['precision']), reverse=True)[0]
print(f"Best model: BaggingClassifier (n_estimators={best_bagging['n_estimators']})")
best_bagging_accuracy = best_bagging['accuracy']
best_bagging_precision = best_bagging['precision']
print(f"accuracy is {best_bagging['accuracy']:.5f}")
print(f"precision is {best_bagging['precision']:.5f}")
print(f"recall is {best_bagging['recall']:.5f}")

Best model: BaggingClassifier (n_estimators=50)
accuracy is 0.88462
precision is 0.88941
recall is 0.88462


In [18]:
best_bagging['model'].fit(X_train, y_train)
y_pred_bag_test = best_bagging['model'].predict(X_test)
best_bagging_model = best_bagging['model']
accuracy_test_bagging = accuracy_score(y_test, y_pred_bag_test)
precision_test_bagging = precision_score(y_test, y_pred_bag_test, average='weighted')
recall_test_bagging = recall_score(y_test, y_pred_bag_test, average='weighted')
print(f"BaggingClassifier (n_estimators={best_bagging['n_estimators']}) test sample:")
print(f"accuracy is {accuracy_score(y_test, y_pred_bag_test):.5f}")
print(f"precision is {precision_score(y_test, y_pred_bag_test, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred_bag_test, average='weighted'):.5f}")

BaggingClassifier (n_estimators=50) test sample:
accuracy is 0.88462
precision is 0.88941
recall is 0.88462


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [19]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=21)
print(cv)

StratifiedKFold(n_splits=5, random_state=21, shuffle=True)


In [20]:
stacking_results = []
for n_splits in [2, 3, 4, 5, 6, 7]:
    for passthrough in [False, True]:
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=21)
        stacking = StackingClassifier(
            estimators=[
                ('svm', svm),
                ('tree', tree),
                ('rf', rfc)
            ],
            final_estimator=OneVsRestClassifier(LogisticRegression(solver='liblinear')),
            cv=cv,
            passthrough=passthrough,
            n_jobs=-1
        )
        stacking.fit(X_train, y_train)
        y_pred_stack = stacking.predict(X_valid)
        stacking_results.append({
            'n_splits': n_splits,
            'passthrough': passthrough,
            'accuracy': accuracy_score(y_valid, y_pred_stack),
            'precision': precision_score(y_valid, y_pred_stack, average='weighted'),
            'recall': recall_score(y_valid, y_pred_stack, average='weighted'),
            'model': stacking
        })
        print(f"StackingClassifier (n_splits={n_splits}, passthrough={passthrough}):")
        print(f"accuracy is {accuracy_score(y_valid, y_pred_stack):.5f}")
        print(f"precision is {precision_score(y_valid, y_pred_stack, average='weighted'):.5f}")
        print(f"recall is {recall_score(y_valid, y_pred_stack, average='weighted'):.5f}")
        print()

StackingClassifier (n_splits=2, passthrough=False):
accuracy is 0.89630
precision is 0.89678
recall is 0.89630

StackingClassifier (n_splits=2, passthrough=True):
accuracy is 0.90370
precision is 0.90619
recall is 0.90370

StackingClassifier (n_splits=3, passthrough=False):
accuracy is 0.89630
precision is 0.89759
recall is 0.89630

StackingClassifier (n_splits=3, passthrough=True):
accuracy is 0.90370
precision is 0.90632
recall is 0.90370

StackingClassifier (n_splits=4, passthrough=False):
accuracy is 0.90370
precision is 0.90570
recall is 0.90370

StackingClassifier (n_splits=4, passthrough=True):
accuracy is 0.91111
precision is 0.91327
recall is 0.91111

StackingClassifier (n_splits=5, passthrough=False):
accuracy is 0.90000
precision is 0.90056
recall is 0.90000

StackingClassifier (n_splits=5, passthrough=True):
accuracy is 0.90000
precision is 0.90217
recall is 0.90000

StackingClassifier (n_splits=6, passthrough=False):
accuracy is 0.90370
precision is 0.90436
recall is 0.903

In [21]:
best_stack = sorted(stacking_results, key=lambda x: (x['accuracy'], x['precision']), reverse=True)[0]
print(f"Лучшая модель: StackingClassifier (n_splits={best_stack['n_splits']}, passthrough={best_stack['passthrough']})")
best_stacking_accuracy = best_stack['accuracy']
best_stacking_precision = best_stack['precision']
print(f"accuracy is {best_stack['accuracy']:.5f}")
print(f"precision is {best_stack['precision']:.5f}")
print(f"recall is {best_stack['recall']:.5f}")

Лучшая модель: StackingClassifier (n_splits=4, passthrough=True)
accuracy is 0.91111
precision is 0.91327
recall is 0.91111


In [22]:
best_stack['model'].fit(X_train, y_train)
y_pred_stack_test = best_stack['model'].predict(X_test)
best_stacking_model = best_stack['model']
accuracy_test_stacking = accuracy_score(y_test, y_pred_stack_test)
precision_test_stacking = precision_score(y_test, y_pred_stack_test, average='weighted')
recall_test_stacking = recall_score(y_test, y_pred_stack_test, average='weighted')
print(f"StackingClassifier (n_splits={best_stack['n_splits']}, passthrough={best_stack['passthrough']}) на тестовой выборке:")
print(f"accuracy is {accuracy_score(y_test, y_pred_stack_test):.5f}")
print(f"precision is {precision_score(y_test, y_pred_stack_test, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred_stack_test, average='weighted'):.5f}")

StackingClassifier (n_splits=4, passthrough=True) на тестовой выборке:
accuracy is 0.90533
precision is 0.90844
recall is 0.90533


## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [23]:
models_comparison = {
    'SVM': {'accuracy': accuracy_svm, 'precision': precision_svm},
    'Decision Tree': {'accuracy': accuracy_tree, 'precision': precision_tree},
    'Random Forest': {'accuracy': accuracy_rfc, 'precision': precision_rfc},
    'Voting Classifier': {'accuracy': best_voting_accuracy, 'precision': best_voting_precision},
    'Bagging Classifier': {'accuracy': best_bagging_accuracy, 'precision': best_bagging_precision},
    'Stacking Classifier': {'accuracy': best_stacking_accuracy, 'precision': best_stacking_precision}
}

best_model_name = None
best_accuracy = 0
best_precision = 0

for model_name, metrics in models_comparison.items():
    if (metrics['accuracy'] > best_accuracy or 
        (metrics['accuracy'] == best_accuracy and metrics['precision'] > best_precision)):
        best_accuracy = metrics['accuracy']
        best_precision = metrics['precision']
        best_model_name = model_name


print(f"Best model name: {best_model_name}")
print(f"Accuracy: {best_accuracy:.5f}")
print(f"Precision: {best_precision:.5f}")


models_comparison_test = {
    'Voting Classifier': {'accuracy': accuracy_test_voting, 'precision': precision_test_voting, 'recall': recall_test_voting},
    'Bagging Classifier': {'accuracy': accuracy_test_bagging, 'precision': precision_test_bagging, 'recall': recall_test_bagging},
    'Stacking Classifier': {'accuracy': accuracy_test_stacking, 'precision': precision_test_stacking, 'recall': recall_test_stacking}
}

best_model_name = None
best_accuracy = 0
best_precision = 0

for model_name, metrics in models_comparison_test.items():
    if (metrics['accuracy'] > best_accuracy or 
        (metrics['accuracy'] == best_accuracy and metrics['precision'] > best_precision)):
        best_accuracy = metrics['accuracy']
        best_precision = metrics['precision']
        best_recall = metrics['recall']
        best_model_name = model_name

print(f"Best model name test sample:{best_model_name}")
print(f"accuracy is {best_accuracy:.5f}")
print(f"precision is {best_precision:.5f}")
print(f"recall is {best_recall:.5f}")

if best_model_name == 'SVM':
    best_model = svm_model
elif best_model_name == 'Decision Tree':
    best_model = tree_model
elif best_model_name == 'Random Forest':
    best_model = rfc_model
elif best_model_name == 'Voting Classifier':
    best_model = best_voting_model
elif best_model_name == 'Bagging Classifier':
    best_model = best_bagging_model
else:  
    best_model = best_stacking_model

y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best, average='weighted', zero_division=0)
recall_best = recall_score(y_test, y_pred_best, average='weighted', zero_division=0)


Best model name: Stacking Classifier
Accuracy: 0.91111
Precision: 0.91327
Best model name test sample:Voting Classifier
accuracy is 0.90533
precision is 0.90881
recall is 0.90533


In [24]:
print(" Day |  All  |  Error | % Error")
error_rates_day = []
for day in range(7):
    day_mask = y_test == day
    total_samples = np.sum(day_mask)
    wrong_predictions = np.sum(y_pred_best[day_mask] != day)
    error_rate = wrong_predictions / total_samples if total_samples > 0 else 0
    error_rates_day.append(error_rate)
    print(f"{day:4} | {total_samples:5} | {wrong_predictions:6} | {error_rate:.3f}")

worst_day = np.argmax(error_rates_day)
print(f"error rates day {worst_day} ({error_rates_day[worst_day]:.3f} error)")

 Day |  All  |  Error | % Error
   0 |    27 |      8 | 0.296
   1 |    55 |      4 | 0.073
   2 |    30 |      2 | 0.067
   3 |    80 |      3 | 0.037
   4 |    21 |      2 | 0.095
   5 |    54 |      7 | 0.130
   6 |    71 |      6 | 0.085
error rates day 0 (0.296 error)


In [25]:
labname_columns = [col for col in X.columns if col.startswith('labname_')]
labname_errors = []

for lab_col in labname_columns:
    lab_mask = X_test[lab_col] == 1
    if lab_mask.sum() > 0:
        total_with_lab = lab_mask.sum()
        wrong_with_lab = np.sum(y_pred_best[lab_mask] != y_test[lab_mask])
        error_rate = wrong_with_lab / total_with_lab
        
        labname_errors.append({
            'labname': lab_col,
            'total_samples': total_with_lab,
            'wrong_predictions': wrong_with_lab,
            'error_rate': error_rate
        })

labname_errors_sorted = sorted(labname_errors, key=lambda x: x['error_rate'], reverse=True)

print("Labname |  All  |  Error | % Error")
for lab in labname_errors_sorted[:5]:
    lab_short = lab['labname'].replace('labname_', '')[:15]
    print(f"{lab_short:7} | {lab['total_samples']:5} | {lab['wrong_predictions']:6} | {lab['error_rate']:.3f}")

Labname |  All  |  Error | % Error
lab03   |     1 |      1 | 1.000
laba04  |    35 |      9 | 0.257
laba04s |    25 |      6 | 0.240
lab05s  |     6 |      1 | 0.167
laba06  |     9 |      1 | 0.111


In [26]:
user_columns = [col for col in X.columns if col.startswith('uid_user_')]
user_errors = []

for user_col in user_columns:
    user_mask = X_test[user_col] == 1
    total_with_user = user_mask.sum()
    if total_with_user > 0:
        wrong_with_user = np.sum(y_pred_best[user_mask] != y_test[user_mask])
        error_rate = wrong_with_user / total_with_user
        
        user_errors.append({
            'user': user_col.replace('uid_user_', ''),
            'total_samples': total_with_user,
            'wrong_predictions': wrong_with_user,
            'error_rate': error_rate
        })

user_errors_sorted = sorted(user_errors, key=lambda x: x['error_rate'], reverse=True)


print(f"User{' ':8}   | {'All':>6} | {'Errors':>6} | {'% Errors':>8}")
for user in user_errors_sorted[:10]:
    print(f"{user['user']:14} | {user['total_samples']:6} | {user['wrong_predictions']:6} | {user['error_rate']:8.3f}")

User           |    All | Errors | % Errors
6              |      4 |      2 |    0.500
17             |      7 |      2 |    0.286
3              |     14 |      3 |    0.214
16             |      5 |      1 |    0.200
18             |      6 |      1 |    0.167
27             |      6 |      1 |    0.167
19             |     19 |      3 |    0.158
2              |     28 |      4 |    0.143
30             |      8 |      1 |    0.125
13             |     17 |      2 |    0.118


In [27]:
joblib.dump(best_model, '../data/best_ensemble_model.joblib')

['../data/best_ensemble_model.joblib']